![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_AlbertForQuestionAnswering.ipynb)

# Import OpenVINO AlbertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting AlbertForQuestionAnswering models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for AlbertForQuestionAnswering from AlbertForQuestionAnswering and they have to be in `Question Answering` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.41.2
!pip install -q --upgrade openvino==2024.1
!pip install -q --upgrade optimum-intel==1.17.0
!pip install -q --upgrade onnx==1.12.0


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openvino-dev 2024.6.0 requires openvino==2024.6.0, but you have openvino 2024.1.0 which is incompatible.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [twmkn9/albert-base-v2-squad2](https://huggingface.co/twmkn9/albert-base-v2-squad2) model from HuggingFace as an example and load it as a `OVModelForQuestionAnswering`, representing an OpenVINO model.
- In addition to the AlbertForQuestionAnswering model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForQuestionAnswering
from transformers import AutoTokenizer

MODEL_NAME = "twmkn9/albert-base-v2-squad2"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Save the OpenVINO model
ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

# Create directory for assets and move the tokenizer files.
# A separate folder is needed for Spark NLP.
!mkdir {EXPORT_PATH}/assets
!mv {EXPORT_PATH}/spiece.model  {EXPORT_PATH}/assets/

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alegp97/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Framework not specified. Using pt to export the model.
Some weights of the model checkpoint at twmkn9/albert-base-v2-squad2 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 46112
drwxrwx---+ 2 alegp97 alegp97     4096 may 12 16:48 assets
-rw-rw----+ 1 alegp97 alegp97      844 may 12 16:48 config.json
-rw-rw----+ 1 alegp97 alegp97 44382412 may 12 16:48 openvino_model.bin
-rw-rw----+ 1 alegp97 alegp97   504762 may 12 16:48 openvino_model.xml
-rw-rw----+ 1 alegp97 alegp97      286 may 12 16:48 special_tokens_map.json
-rw-rw----+ 1 alegp97 alegp97     1264 may 12 16:48 tokenizer_config.json
-rw-rw----+ 1 alegp97 alegp97  2274146 may 12 16:48 tokenizer.json


In [4]:
!ls -l {EXPORT_PATH}/assets

total 748
-rw-rw----+ 1 alegp97 alegp97 760289 may 12 16:48 spiece.model


## Import and Save AlbertForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [5]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 6.0.0
setup Colab for PySpark 3.2.3 and Spark NLP 6.0.0

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()
    

:: loading settings :: url = jar:file:/opt/spark-3.5.1/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/alegp97/.ivy2/cache
The jars for the packages stored in: /home/alegp97/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5c560117-7ef8-4eb5-9c14-3d0ba55bd03e;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.0.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in cent

- Let's use `loadSavedModel` functon in `AlbertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [8]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = AlbertForQuestionAnswering.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

25/05/12 16:56:33 WARN NativeLibrary: Failed to load library null: java.lang.UnsatisfiedLinkError: Can't load library: /tmp/openvino-native4571324895940381217/libtbb.so.2
25/05/12 16:56:33 WARN NativeLibrary: Failed to load library null: java.lang.UnsatisfiedLinkError: /tmp/openvino-native4571324895940381217/libopenvino.so: libtbb.so.12: cannot open shared object file: No such file or directory
25/05/12 16:56:33 WARN NativeLibrary: Failed to load library null: java.lang.UnsatisfiedLinkError: /tmp/openvino-native4571324895940381217/libinference_engine_java_api.so: libopenvino.so.2410: cannot open shared object file: No such file or directory
25/05/12 16:56:33 ERROR OpenvinoWrapper$: Could not initialize OpenVINO Core. Please make sure the jsl-openvino JAR is loaded and Intel oneTBB is installed.
(See https://www.intel.com/content/www/us/en/docs/onetbb/get-started-guide/2021-12/overview.html)


Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.annotators.classifier.dl.AlbertForQuestionAnswering.loadSavedModel.
: java.lang.UnsatisfiedLinkError: 'long org.intel.openvino.Core.GetCore()'
	at org.intel.openvino.Core.GetCore(Native Method)
	at org.intel.openvino.Core.<init>(Core.java:22)
	at com.johnsnowlabs.ml.openvino.OpenvinoWrapper$.liftedTree1$1(OpenvinoWrapper.scala:79)
	at com.johnsnowlabs.ml.openvino.OpenvinoWrapper$.<init>(OpenvinoWrapper.scala:78)
	at com.johnsnowlabs.ml.openvino.OpenvinoWrapper$.<clinit>(OpenvinoWrapper.scala)
	at com.johnsnowlabs.nlp.annotators.classifier.dl.ReadAlbertForQuestionAnsweringDLModel.loadSavedModel(AlbertForQuestionAnswering.scala:399)
	at com.johnsnowlabs.nlp.annotators.classifier.dl.ReadAlbertForQuestionAnsweringDLModel.loadSavedModel$(AlbertForQuestionAnswering.scala:360)
	at com.johnsnowlabs.nlp.annotators.classifier.dl.AlbertForQuestionAnswering$.loadSavedModel(AlbertForQuestionAnswering.scala:419)
	at com.johnsnowla

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your AlbertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 44592
-rw-r--r-- 1 root root 44894313 Sep  8 20:25 albert_classification_openvino
-rw-r--r-- 1 root root   760289 Sep  8 20:25 albert_spp
drwxr-xr-x 2 root root     4096 Sep  8 20:25 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [ ]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = AlbertForQuestionAnswering.load("./{}_spark_nlp_openvino".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `AlbertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
